<a href="https://colab.research.google.com/github/Muthumaha-S/Alzheimer_Prediction/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from werkzeug.utils import secure_filename

# Flask app initialization
app = Flask(__name__)

# Set upload folder
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure uploads folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Load the trained model
MODEL_PATH = '/content/drive/MyDrive/alzheimers_prediction/alzheimer_prediction_model.h5'
model = load_model(MODEL_PATH)

# Categories should match the training labels
CATEGORIES = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]

IMG_SIZE = 128  # Must be the same size as used in training


# Convert grayscale image to RGB (3 channels)
def convert_to_rgb(image):
    return np.repeat(image, 3, axis=-1)


# Home page
@app.route('/')
def home():
    return render_template('index.html')


# Prediction route
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    if file:
        filename = secure_filename(file.filename)
        file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(file_path)

        # Process the image
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0  # Normalize
        img = np.expand_dims(img, axis=-1)  # Add channel dimension (128, 128, 1)
        img = np.expand_dims(img, axis=0)   # Add batch dimension (1, 128, 128, 1)

        # Convert to RGB (128, 128, 3)
        img_rgb = convert_to_rgb(img)

        # Make prediction
        pred = model.predict(img_rgb)
        class_idx = np.argmax(pred, axis=1)[0]
        prediction = CATEGORIES[class_idx]

        # Pass the file path and prediction to the result page
        image_url = url_for('static', filename=f'uploads/{filename}')
        # Save uploaded images in static/uploads folder if necessary
        static_upload_path = os.path.join('static/uploads', filename)
        os.makedirs(os.path.dirname(static_upload_path), exist_ok=True)
        cv2.imwrite(static_upload_path, cv2.imread(file_path))

        return render_template('result.html', prediction=prediction, image_url=image_url)

    return redirect(url_for('home'))


# Run the app
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
